<a href="https://colab.research.google.com/github/yeialgov/MAYC_ML_Revenue_Prediction/blob/patch-1/Classification%20and%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download the libraries
import pandas as pd
from pandas import Series,DataFrame
import warnings
from copy import deepcopy
import os
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import random
%matplotlib inline

In [0]:
from sklearn.preprocessing import LabelEncoder

# Getting the data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd '/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean/'

/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean


In [0]:
all_data = pd.read_pickle('all_data.pkl')


In [6]:
all_data.isnull().sum().sum()

0

In [7]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 32 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   totals.sessionQualityDim                      int64         
 1   totals.totalTransactionRevenue                float64       
 2   device.deviceCategory                         object        
 3   totals.timeOnSite                             int64         
 4   totals.bounces                                int64         
 5   geoNetwork.country                            object        
 6   visitStartTime                                int64         
 7   totals.transactions                           int64         
 8   totals.pageviews                              int64         
 9   date                                          datetime64[ns]
 10  customDimension.value                         object        
 11  dataset                  

In [0]:
'''
columns to hotcode

channelGrouping                               object  unique values:  8
date                                          int64  - convert to date time
fullVisitorId                                 object - organisers insist to be left as str
device.browser                                object - unique values:  129
device.operatingSystem                        object - unique values:  24
device.isMobile                               bool   - unique values:  2
device.deviceCategory                         object - unique values:  3
geoNetwork.continent                          object - unique values:  6
geoNetwork.subContinent                       object - unique values:  23
geoNetwork.country                            object - unique values:  228
geoNetwork.region                             object - unique values:  284
geoNetwork.metro                              object - unique values:  123
geoNetwork.city                               object - unique values:  956
geoNetwork.networkDomain                      object - unique values:  41982
totals.sessionQualityDim                      object - convert to interger
totals.timeOnSite                             object - convert to interger
trafficSource.source                          object - unique values:  345
trafficSource.medium                          object - unique values:  7
trafficSource.isTrueDirect                    bool   - unique values:  2 
trafficSource.adwordsClickInfo.adNetworkType  object - unique values:  3
trafficSource.adwordsClickInfo.isVideoAd      bool   - unique values:  2 
customDimension.value                         object - unique values:  5
'''

# Model to search the visitors who made transactions

## Function to define Target Column

In [0]:
def is_transaction(n):
  if n == 0    :    return 0
  else         :    return 1

all_data['is_transaction']=all_data['totals.transactions'].apply(is_transaction)

In [9]:
all_data['is_transaction'].value_counts()

0    2085062
1      24864
Name: is_transaction, dtype: int64

## Encoding Categorical Values

In [0]:
TARGET = 'is_transaction'
CAT_COLS = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS = [c for c in all_data.columns if c not in CAT_COLS and c != TARGET]


In [0]:
CAT_COLS.remove('date')
CAT_COLS.remove('fullVisitorId')
CAT_COLS

In [0]:
CONT_COLS.remove('date')

In [13]:
CONT_COLS.remove('totals.transactions')
CONT_COLS.remove('totals.totalTransactionRevenue')
CONT_COLS

['totals.sessionQualityDim',
 'totals.timeOnSite',
 'totals.bounces',
 'visitStartTime',
 'totals.pageviews',
 'date',
 'totals.newVisits',
 'visitNumber',
 'trafficSource.isTrueDirect',
 'visitId',
 'device.isMobile',
 'totals.hits',
 'trafficSource.adwordsClickInfo.isVideoAd']

In [15]:
M1_data = deepcopy(all_data[CAT_COLS + CONT_COLS + [TARGET]])

M1_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 31 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   device.deviceCategory                         object        
 1   geoNetwork.country                            object        
 2   customDimension.value                         object        
 3   dataset                                       object        
 4   geoNetwork.metro                              object        
 5   geoNetwork.continent                          object        
 6   trafficSource.medium                          object        
 7   device.operatingSystem                        object        
 8   geoNetwork.networkDomain                      object        
 9   channelGrouping                               object        
 10  trafficSource.adwordsClickInfo.adNetworkType  object        
 11  device.browser           

In [0]:
leb = LabelEncoder()
for col in CAT_COLS:
    M1_data[col] = leb.fit_transform(M1_data[col])

In [23]:
M1_data['visitId'].size

2109926

In [24]:
M1_data['visitId'].nunique()

2058981

## Split Train and Test

In [25]:
M1_data['dataset'].value_counts()

1    1708337
0     401589
Name: dataset, dtype: int64

In [0]:
train = M1_data[M1_data['dataset']==1]
test = M1_data[M1_data['dataset']==0]
train.drop('dataset',axis=1,inplace=True)
test.drop('dataset',axis=1, inplace=True)

In [0]:
CAT_COLS.remove('dataset')

In [28]:
train.shape, test.shape

((1708337, 30), (401589, 30))

## Model

In [0]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
rmse = lambda true, pred: mse(true, pred) ** 0.5

In [0]:
x_train = train[CAT_COLS + CONT_COLS]
x_test = test[CAT_COLS + CONT_COLS]
y_train = train[[TARGET]]
y_test = test[[TARGET]]

In [42]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1708337, 28), (401589, 28), (1708337, 1), (401589, 1))

In [43]:
x_train.columns

Index(['device.deviceCategory', 'geoNetwork.country', 'customDimension.value',
       'geoNetwork.metro', 'geoNetwork.continent', 'trafficSource.medium',
       'device.operatingSystem', 'geoNetwork.networkDomain', 'channelGrouping',
       'trafficSource.adwordsClickInfo.adNetworkType', 'device.browser',
       'geoNetwork.subContinent', 'geoNetwork.region', 'trafficSource.source',
       'geoNetwork.city', 'fullVisitorId', 'totals.sessionQualityDim',
       'totals.timeOnSite', 'totals.bounces', 'visitStartTime',
       'totals.pageviews', 'totals.newVisits', 'visitNumber',
       'trafficSource.isTrueDirect', 'visitId', 'device.isMobile',
       'totals.hits', 'trafficSource.adwordsClickInfo.isVideoAd'],
      dtype='object')

In [0]:
model = RandomForestClassifier( 
    n_estimators=100, # Hyperparameter 1
    max_depth=10,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model = model.fit(x_train, y_train)

In [0]:
def get_error(pred, true):
    print(f'RMSE:       {rmse(true, pred)}')
    print(f'MAE:        {mae(true, pred)}')
    print(f'r2:         {r2(true, pred)}')
    print(f'accuracy:   {accuracy(true, pred)}')

In [120]:
get_error(pred = trained_model.predict(x_test),
          true = y_test)

RMSE:       0.12455269216383845
MAE:        0.015513373125259905
r2:         -0.003865498894937147
accuracy:   0.9844866268747401


In [0]:
pred_M1 = trained_model.predict(x_test)


In [122]:
mylist = list(set(pred_M1))
mylist

[0, 1]

In [124]:
T_predicted = pred_M1[pred_M1!=0]
T_predicted.shape

(745,)

In [0]:
train_id = train["fullVisitorId"].values
test_id = test["fullVisitorId"].values

# Model to calculate transaction Revenue

## Encoding Categorical Values - M2

In [0]:
TARGET_M2 = 'totals.totalTransactionRevenue'
CAT_COLS_M2 = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS_M2 = [c for c in all_data.columns if c not in CAT_COLS_M2 and c != TARGET_M2]


In [49]:
#CAT_COLS_M2.remove('date')
#CAT_COLS_M2.remove('fullVisitorId')
CAT_COLS_M2

['device.deviceCategory',
 'geoNetwork.country',
 'customDimension.value',
 'dataset',
 'geoNetwork.metro',
 'geoNetwork.continent',
 'trafficSource.medium',
 'device.operatingSystem',
 'geoNetwork.networkDomain',
 'channelGrouping',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'device.browser',
 'geoNetwork.subContinent',
 'geoNetwork.region',
 'trafficSource.source',
 'geoNetwork.city',
 'fullVisitorId']

In [50]:
#CONT_COLS.remove('totals.transactions')
#CONT_COLS.remove('totals.transactionRevenue')
#CONT_COLS.remove('totals.totalTransactionRevenue')
CONT_COLS_M2

['totals.sessionQualityDim',
 'totals.timeOnSite',
 'totals.bounces',
 'visitStartTime',
 'totals.transactions',
 'totals.pageviews',
 'date',
 'totals.newVisits',
 'visitNumber',
 'trafficSource.isTrueDirect',
 'visitId',
 'device.isMobile',
 'totals.hits',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'is_transaction']

In [51]:
M2_data = deepcopy(all_data[CAT_COLS_M2 + CONT_COLS_M2 + [TARGET_M2]])

M2_data.shape

(2109926, 33)

In [0]:
leb = LabelEncoder()
for col in CAT_COLS_M2:
    M2_data[col] = leb.fit_transform(M2_data[col])

## Split Train and Test - M2

In [0]:
train_M2 = M2_data[M2_data['dataset']==1]
test_M2 = M2_data[M2_data['dataset']==0]
train_M2.drop('dataset',axis=1,inplace=True)
test_M2.drop('dataset',axis=1, inplace=True)

In [0]:
CAT_COLS_M2.remove('dataset')

In [56]:
train_M2.shape, test_M2.shape

((1708337, 32), (401589, 32))

But the training of the transaction revenue is made just with the data from visitors that made at least one transaction

In [0]:
T_train_M2 = train_M2[train_M2['is_transaction']==1]
T_test_M2 = test_M2[test_M2['is_transaction']==1]
T_train_M2.drop('is_transaction',axis=1,inplace=True)
T_test_M2.drop('is_transaction',axis=1, inplace=True)

In [0]:
CONT_COLS_M2.remove('date')
CONT_COLS_M2.remove('is_transaction')

In [59]:
T_train_M2.shape, T_test_M2.shape

((18559, 31), (6305, 31))

In [61]:
T_train_M2.columns == T_test_M2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [71]:
T_test_M2.columns

Index(['device.deviceCategory', 'geoNetwork.country', 'customDimension.value',
       'geoNetwork.metro', 'geoNetwork.continent', 'trafficSource.medium',
       'device.operatingSystem', 'geoNetwork.networkDomain', 'channelGrouping',
       'trafficSource.adwordsClickInfo.adNetworkType', 'device.browser',
       'geoNetwork.subContinent', 'geoNetwork.region', 'trafficSource.source',
       'geoNetwork.city', 'fullVisitorId', 'totals.sessionQualityDim',
       'totals.timeOnSite', 'totals.bounces', 'visitStartTime',
       'totals.transactions', 'totals.pageviews', 'date', 'totals.newVisits',
       'visitNumber', 'trafficSource.isTrueDirect', 'visitId',
       'device.isMobile', 'totals.hits',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'totals.totalTransactionRevenue'],
      dtype='object')

## Model - M2

In [0]:
x_train_M2 = T_train_M2[CAT_COLS_M2 + CONT_COLS_M2]
x_test_M2 = T_test_M2[CAT_COLS_M2 + CONT_COLS_M2]
y_train_M2 = T_train_M2[[TARGET_M2]]
y_test_M2 = T_test_M2[[TARGET_M2]]

In [73]:
x_train_M2.shape, x_test_M2.shape, y_train_M2.shape, y_test_M2.shape

((18559, 29), (6305, 29), (18559, 1), (6305, 1))

In [66]:
x_train_M2.head(5)

,device.deviceCategory,geoNetwork.country,customDimension.value,geoNetwork.metro,geoNetwork.continent,trafficSource.medium,device.operatingSystem,geoNetwork.networkDomain,channelGrouping,trafficSource.adwordsClickInfo.adNetworkType,device.browser,geoNetwork.subContinent,geoNetwork.region,trafficSource.source,geoNetwork.city,fullVisitorId,totals.sessionQualityDim,totals.timeOnSite,totals.bounces,visitStartTime,totals.transactions,totals.pageviews,date,totals.newVisits,visitNumber,trafficSource.isTrueDirect,visitId,device.isMobile,totals.hits,trafficSource.adwordsClickInfo.isVideoAd
433,0,219,4,78,2,5,7,0,4,0,55,12,304,117,664,1544852,13,261,0,1508200705,1,9,2017-10-16,0,4,True,1508200705,False,9.0,True
550,0,219,4,64,2,0,22,0,6,0,55,12,74,0,432,598266,38,285,0,1508192636,1,12,2017-10-16,0,11,False,1508192636,False,15.0,True
555,1,219,4,104,2,5,25,34258,4,0,102,12,74,117,621,253734,42,1044,0,1508162218,1,15,2017-10-16,0,6,True,1508162218,True,15.0,True
597,0,219,4,104,2,0,7,0,6,0,55,12,74,0,621,1162248,77,514,0,1508189401,1,16,2017-10-16,0,17,True,1508189401,False,18.0,True
619,0,219,4,78,2,0,7,0,6,0,55,12,304,0,664,1276019,62,487,0,1508190484,1,20,2017-10-16,1,1,False,1508190484,False,21.0,True


In [67]:
y_train_M2.head(5)

,totals.totalTransactionRevenue
433,17190000.0
550,13000000.0
555,65300000.0
597,25230000.0
619,16310000.0


In [0]:
model_M2 = RandomForestRegressor( 
    n_estimators=25, # Hyperparameter 1
    max_depth=2,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model_M2 = model_M2.fit(x_train_M2, y_train_M2)

In [0]:
def get_error_regression(pred, true):
    print(f'RMSE:       {rmse(true, pred)}')
    print(f'MAE:        {mae(true, pred)}')
    print(f'r2:         {r2(true, pred)}')

In [87]:
get_error_regression(pred = trained_model_M2.predict(x_test_M2),
          true = y_test_M2)

RMSE:       505274468.6432537
MAE:        134320421.89680147
r2:         0.06662577370784417


In [99]:
pred_M2 = trained_model_M2.predict(x_test_M2)
pred_M2.shape

(6305,)

# build a submission file

In [0]:
train_id = train["fullVisitorId"].values
test_id = test["fullVisitorId"].values

In [0]:
sub_df = pd.DataFrame({"fullVisitorId":test_id})
pred_M2[pred_M2<0] = 0
#pred_test[pred_test<0] = 0
#sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
#sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
#sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
#sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
#sub_df.to_csv("baseline_lgb.csv", index=False)

In [104]:
pred_M1.shape, pred_M2.shape

((401589,), (6305,))

In [101]:
sub_df.shape

(401589, 1)